In [ ]:
!pip install transformers
!pip install datasets
!pip install jiwer
!pip install torchaudio
!pip install evaluate

In [ ]:
!pip install torchaudio --force-reinstall
!pip install librosa --upgrade

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import os
import pandas as pd
import torchaudio
import torch
from evaluate import load

# Parameters
language_code = 'ar'
language_name = 'arabic'
new_output_models_dir = f"./Model-Finetuned-v17"
test_data_path = "./Dataset/test"

# Load WER metric, processor, and model
wer_metric = load("wer", trust_remote_code=True)
processor = Wav2Vec2Processor.from_pretrained(new_output_models_dir)
model = Wav2Vec2ForCTC.from_pretrained(new_output_models_dir)
model.to("cpu")

# Function to transcribe audio file
def transcribe(audio_file_path):
    waveform, sample_rate = torchaudio.load(audio_file_path)
    input_values = processor(waveform.squeeze().numpy(), return_tensors="pt", padding="longest").input_values.to("cpu")
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription

# Generate transcription for all test files
submission_data = []
for audio_file in os.listdir(test_data_path):
    if audio_file.endswith('.wav'):
        audio_file_path = os.path.join(test_data_path, audio_file)
        transcription = transcribe(audio_file_path)
        audio_file_id = os.path.splitext(audio_file)[0]
        submission_data.append((audio_file_id, transcription))

# Create DataFrame
submission_df = pd.DataFrame(submission_data, columns=['audio', 'transcript'])

# Save to CSV
submission_file_path = os.path.join(new_output_models_dir, 'submission_17.csv')
submission_df.to_csv(submission_file_path, index=False)

print(f"Submission file saved to {submission_file_path}")


c:\Users\Alex\Downloads\MTC-AIC2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be har